In [3]:
pip install opencv-python



  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)


In [6]:
pip install tensorflow

  Using cached tensorflow-2.17.0-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.12.1-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.4.1-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached grpcio-1.67.0-cp311-cp311-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.17.1-py3-none-any.whl.metadata (1.6 kB)

In [4]:
#Data Preparation

import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

data_path = 'Downloads/sign_lang'
labels = {'hello': 0, 'thanks': 1, 'please': 2}  # Example labels
images = []
targets = []

for label in labels:
    sign_path = os.path.join(data_path, label)
    for image_file in os.listdir(sign_path):
        img_path = os.path.join(sign_path, image_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))
        images.append(img)
        targets.append(labels[label])

images = np.array(images) / 255.0
targets = np.array(targets)

X_train, X_val, y_train, y_val = train_test_split(images, targets, test_size=0.2, random_state=42)

In [7]:
#Model Creation
#Build a CNN for image classification:


import tensorflow as tf
from tensorflow.keras import layers, models

def create_sign_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(len(labels), activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

sign_model = create_sign_model()


C:\Users\harrr\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
#Model Training
#Train the model:


sign_model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))
sign_model.save('sign_model.h5')

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 308ms/step - accuracy: 0.3688 - loss: 1.2271 - val_accuracy: 0.3636 - val_loss: 1.4088
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.4292 - loss: 1.2040 - val_accuracy: 0.2727 - val_loss: 1.1397
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.3708 - loss: 1.0514 - val_accuracy: 0.2727 - val_loss: 1.1465
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.5271 - loss: 0.9833 - val_accuracy: 0.2727 - val_loss: 1.2456
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.5979 - loss: 0.9154 - val_accuracy: 0.2727 - val_loss: 1.1949
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.6417 - loss: 0.7900 - val_accuracy: 0.1818 - val_loss: 1.5928
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.7229 - loss: 0.7066 - val_accuracy: 0.2727 - val_loss: 1.6509
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.7167 - loss: 0.6093 - val_accuracy: 0.1818 - val_loss:

In [8]:


#GUI Development with Time-based Operation
#Create a GUI using tkinter with features for image upload and real-time video processing, and include the time-based operational restriction:



In [10]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from datetime import datetime
import cv2
from PIL import Image, ImageTk

def within_operational_hours():
    current_time = datetime.now().time()
    return current_time >= datetime.strptime('18:00', '%H:%M').time() and current_time <= datetime.strptime('22:00', '%H:%M').time()

def check_time_and_operate(func):
    def wrapper(*args, **kwargs):
        if within_operational_hours():
            return func(*args, **kwargs)
        else:
            messagebox.showwarning("Operation Not Allowed", "Operations are only allowed between 6 PM and 10 PM")
    return wrapper

@check_time_and_operate
def upload_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        img = Image.open(file_path)
        img = img.resize((128, 128), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        panel.config(image=img)
        panel.image = img
        predict_image(file_path)

@check_time_and_operate
def predict_image(file_path):
    img = cv2.imread(file_path)
    img = cv2.resize(img, (128, 128)) / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = sign_model.predict(img)
    predicted_label = np.argmax(prediction[0])
    label.config(text=f"Predicted Sign: {list(labels.keys())[predicted_label]}")

@check_time_and_operate
def predict_video():
    cap = cv2.VideoCapture(0)
    while within_operational_hours() and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        img = cv2.resize(frame, (128, 128)) / 255.0
        img = np.expand_dims(img, axis=0)
        prediction = sign_model.predict(img)
        predicted_label = np.argmax(prediction[0])
        cv2.putText(frame, f"Predicted: {list(labels.keys())[predicted_label]}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow("Sign Language Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Create the main window
root = tk.Tk()
root.title("Sign Language Recognition")

# Create and pack widgets
upload_btn = tk.Button(root, text="Upload Image", command=upload_image)
upload_btn.pack(pady=20)

video_btn = tk.Button(root, text="Real-time Video", command=predict_video)
video_btn.pack(pady=20)

detect_btn = tk.Button(root, text="Detect Sign Language", command=check_time_and_operate(predict_video))
detect_btn.pack(pady=20)

panel = tk.Label(root)
panel.pack(pady=20)

label = tk.Label(root, text="")
label.pack(pady=20)

# Run the application
root.mainloop()

In [9]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
import cv2
from PIL import Image, ImageTk

def upload_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        img = Image.open(file_path)
        img = img.resize((128, 128), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        panel.config(image=img)
        panel.image = img
        predict_image(file_path)

def predict_image(file_path):
    img = cv2.imread(file_path)
    img = cv2.resize(img, (128, 128)) / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = sign_model.predict(img)
    predicted_label = np.argmax(prediction[0])
    label.config(text=f"Predicted Sign: {list(labels.keys())[predicted_label]}")

def predict_video():
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        img = cv2.resize(frame, (128, 128)) / 255.0
        img = np.expand_dims(img, axis=0)
        prediction = sign_model.predict(img)
        predicted_label = np.argmax(prediction[0])
        cv2.putText(frame, f"Predicted: {list(labels.keys())[predicted_label]}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow("Sign Language Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Create the main window
root = tk.Tk()
root.title("Sign Language Recognition")

# Create and pack widgets
upload_btn = tk.Button(root, text="Upload Image", command=upload_image)
upload_btn.pack(pady=20)

video_btn = tk.Button(root, text="Real-time Video", command=predict_video)
video_btn.pack(pady=20)

panel = tk.Label(root)
panel.pack(pady=20)

label = tk.Label(root, text="")
label.pack(pady=20)

# Run the application
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━